In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

In [2]:
df2 = get_fresh_query_result("https://internal-redash.federate.frubana.com/",179153,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [3]:
query="""
SELECT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    cat.parent_description as category,
    CASE WHEN su.source_parent_id = 0 THEN su.source_id ELSE su.source_parent_id END AS card_id,
    --dp.card_id,
    nvl(nullif(su.description_parent,'No value'),su.description) AS product_name,
    --dp.card_description AS product_name,
    sum((fs.product_price * fs.product_quantity_x_step_unit)+fs.product_tax_iva) / nullif(sum(fs.product_quantity_x_step_unit),0) as product_pricee,
    sum((fs.product_price_discount * fs.product_quantity_x_step_unit)+fs.product_tax_iva) / nullif(sum(fs.product_quantity_x_step_unit),0) as product_price_discountt,
    1-(product_price_discountt/product_pricee) AS dct,
    AVG(pb.min_price) AS benchmark,
    LISTAGG(DISTINCT
    CASE
        WHEN (fsd.adjustment_reason LIKE 'BIG LIQUIDATION%') THEN 'LIQUIDATION MAR'
        WHEN (LOWER(fsd.adjustment_reason) ILIKE '%merma%') THEN 'MERMA'
        WHEN ( (LOWER(fsd.adjustment_reason) ILIKE '%acmkt%') OR (LOWER(fsd.adjustment_reason) ILIKE '%kof%') OR (LOWER(fsd.adjustment_reason) ILIKE '%campana%') ) THEN 'DCTO. PROVEEDOR'
        WHEN LOWER(fsd.adjustment_reason) ILIKE 'hooK%' THEN 'HOOK'
        WHEN LOWER(fsd.adjustment_reason) ILIKE '%kam%' THEN 'KAM'
        WHEN fsd.adjustment_reason ILIKE '%\_NB\_%' THEN 'SUPER DISCOUNTS'
        WHEN (fsd.adjustment_reason ILIKE '%\_all_Spend-BMS_%') OR (LOWER(fsd.adjustment_reason) ILIKE 'sh\_%') THEN 'SUPER HOOKS'
        WHEN (fsd.adjustment_reason ILIKE '%EXTRA\_INV%')THEN 'ADDITIONAL INVESTMENT'
        WHEN fsd.adjustment_reason IS NULL THEN 'NO DCT'
    ELSE 'OTHERS' END, ',') AS responsable
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id  
INNER JOIN dpr_product_pricing.obt_benchmark_product_prices pb ON pb.source_product_id = dp.source_id AND DATE(fs.order_submitted_date) = pb.benchmark_date
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.product_quantity_x_step_unit > 0
    AND DATE(fs.order_submitted_date) >= DATE_TRUNC('month', date(getdate()) - interval '0 month')
GROUP BY 1,2,3,4,5
"""
df = run_read_dwd_query(query)
df[["product_pricee","product_price_discountt","dct","benchmark"]] = df[["product_pricee","product_price_discountt","dct","benchmark"]].astype(float)

In [4]:
df

,region_code,submit_date,category,card_id,product_name,product_pricee,product_price_discountt,dct,benchmark,responsable
0,BAQ,2023-09-01,Abarrotes,-302628,Harina Precocida Blanca La Nieve x 500gr Bolsa...,1901.1295,1898.3613,0.0015,1700.00,"SUPER HOOKS,SUPER DISCOUNTS,NO DCT"
1,BAQ,2023-09-01,"Carne, pollo & pescados",145410,Pechuga Mercapollo Bolsa con 1 unidad (peso va...,13700.0000,13700.0000,0.0000,12500.00,NO DCT
2,BAQ,2023-09-01,"Carne, pollo & pescados",488680,Pernil Piernas de Pollo Campollo Bolsa con 10...,8450.0000,8450.0000,0.0000,8000.00,NO DCT
3,BAQ,2023-09-01,Congelados,516618,Mix de Verduras Calypso x 1kg Congelado - Bols...,10800.0000,10800.0000,0.0000,9900.00,NO DCT
4,BAQ,2023-09-01,Despensa,339348,Pasta Concha Doria x 1000gr Bolsa x 1000g Desd...,7127.6600,7127.6600,0.0000,6500.00,NO DCT
...,...,...,...,...,...,...,...,...,...,...
64499,VCP,2023-09-26,Mercearia,539630,"Molho Inglês Cepêra 1,01L Pet de 1,01L A parti...",6.3100,6.3100,0.0000,6.50,NO DCT
64500,VCP,2023-09-26,Mercearia,539675,"Molho Barbecue Cepêra 3,5Kg Galão de 3,5Kg A p...",35.3400,35.3400,0.0000,38.90,NO DCT
64501,VCP,2023-09-26,Mercearia,623838,Vinagre De Álcool Colorido Castelo 5L Galão 5L...,11.9125,11.9125,0.0000,10.90,NO DCT
64502,VCP,2023-09-26,Mercearia,625317,Café Brasileiro Tradicional Pouch 500g Pacote ...,14.0500,12.2200,0.1303,11.99,DCTO. PROVEEDOR


In [5]:
# Set multi-index using 'ID' and 'Date' columns
df.set_index(['card_id', 'submit_date'], inplace=True)
df2.set_index(['card_id', 'fecha'], inplace=True)

# Use the combine_first method to replace values in df1 with values from df2
df['benchmark'] = df['benchmark'].combine_first(df2['benchmark'])

# Reset the index to convert the multi-index back to columns
df.reset_index(inplace=True)

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexes/multi.py:3559: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result = lib.fast_unique_multiple([self._values, rvals], sort=sort)


In [6]:
df['gpi'] = df.product_pricee/df.benchmark
df['npi'] = df.product_price_discountt/df.benchmark

In [7]:
# Sort the DataFrame by 'card_id' and 'submit_date'
df.sort_values(by=['card_id', 'submit_date'], inplace=True)

# Calculate the percentage difference
df['delta_price'] = df.groupby('card_id')['product_pricee'].pct_change()# * 100
df['delta_dct'] = df.groupby('card_id')['dct'].diff()# * 100
df['delta_gpi'] = df.groupby('card_id')['gpi'].pct_change()# * 100
df['delta_npi'] = df.groupby('card_id')['npi'].pct_change()# * 100
df['delta_bench'] = df.groupby('card_id')['benchmark'].pct_change()# * 100

# Add a new column 'responsable_combined' that combines yesterday's and today's responsable values
df['responsable_combined'] = df.groupby('card_id')['responsable'].shift(1) + "  ||  " + df['responsable']
df['price_combined'] = df.groupby('card_id')['product_pricee'].shift(1).fillna('').astype(str) + "  ||  " + df['product_pricee'].astype(str)

# Reset the index (optional)
df.reset_index(drop=True, inplace=True)

In [8]:
aaa = df.loc[(~df.category.isin(['Frutas & verduras','Frutas e verduras'])) & (df.delta_bench<0.05) & (df.delta_bench>-0.05),['region_code','submit_date','card_id','product_name','responsable_combined','product_pricee','price_combined','dct','gpi','npi','delta_price','delta_dct','delta_gpi','delta_npi','delta_bench']].dropna()

In [39]:
abs(aaa.delta_price).mean() + 1*abs(aaa.delta_price).mean()

0.025157942285750582

In [40]:
aaa.loc[(abs(aaa.delta_price)>0.025)].to_excel("aa.xlsx") #head(20)#.region_code.value_counts()
# | (abs(aaa.delta_gpi)>2)